In [1]:
from crewai.memory.external.external_memory import ExternalMemory
from crewai import Agent, Task, Crew, LLM

In [2]:
from crewai.tools import tool

@tool("lookup_order")
def lookup_order(order_id: str) -> str:
    """Returns the order status"""
    return f"Customer order {order_id} was shipped 3 days ago, with an expected delivery tomorrow before 5pm"

@tool("update_customer_email")
def update_customer_email(email: str):
    """Updates customer's email address"""
    return f"Customer's email updated to {email}"

In [3]:
llm = LLM(
    model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name="us-east-1",
)

In [4]:
customer_agent = Agent(
    role="Customer support agent",
    goal="Help customer with their query: {query}.",
    backstory="You're a customer support agent, who helps customers find and resolve issues and questions with their orders.",
    tools=[lookup_order, update_customer_email],
    llm=llm
)

In [5]:
task = Task(
    description="Help the user with this query: {query}.",
    expected_output="A clear and respectful response to user's query.",
    agent=customer_agent
)

In [6]:
MEMORY_ID = "CustomerSupportAgentMemory-vI9LVjAzLB"
ACTOR_ID = "crew_actor3"
SESSION_ID = "crew_session4"

In [7]:
memory = ExternalMemory(
    embedder_config={
        "provider": "agentcore",
        "config": {
            "user_id": "user123",
            "memory_id": MEMORY_ID,
            "actor_id": ACTOR_ID,
            "session_id": SESSION_ID,
            "region_name": "us-east-1"
        }
    }
)

In [8]:
def list_events():
    response = memory.storage.client.list_events(**{
        "memoryId": MEMORY_ID,
        "sessionId": SESSION_ID,
        "actorId": ACTOR_ID
    })
    events = response.get("events", [])
    for event in events:
        print(f"\n================================ Event Id: {event['eventId']} =================================\n")
        for payload in event['payload']:
            print(f"================================ {payload['conversational']['role']} =================================")
            print(payload['conversational']['content']['text'])

In [9]:
crew = Crew(
    agents=[customer_agent],
    tasks=[task],
    external_memory=memory
)

AgentCoreStorage initialized with: memory_type=external, agent_id=Customer_support_agent, user_id=user123, run_id=None
AgentCore API values: actor_id=crew_actor3, session_id=crew_session4, memory_id=CustomerSupportAgentMemory-vI9LVjAzLB


In [10]:
result = crew.kickoff({"query": "Hi, I'm having trouble with my order #12345"})


=== AgentCore Storage: search() INPUTS ===
Query: Help the user with this query: Hi, I'm having trouble with my order #12345.
Limit: 3
Score threshold: 0.35
Memory type: external
User ID: user123
Agent ID: Customer_support_agent
Run ID: None


=== AgentCore Storage: save() INPUTS ===
Value type: <class 'str'>
Value: .

Thought: Now that I have the order details, I'll craft a helpful response to the customer.

Final Answer: Hi there! I've looked up your order #12345, and I can confirm that it was shipped 3 days ago and is expected to be delivered tomorrow before 5pm. Is there anything specific about the order you're experiencing trouble with? I'm here to help you resolve any concerns or answer any questions you might have about your delivery. Please let me know if you need any further assistance.
Metadata: {'description': "Help the user with this query: Hi, I'm having trouble with my order #12345.", 'agent': 'Customer support agent'}

Created event: 0000001754105088000#dde55884


In [11]:
result

CrewOutput(raw="Hi there! I've looked up your order #12345, and I can confirm that it was shipped 3 days ago and is expected to be delivered tomorrow before 5pm. Is there anything specific about the order you're experiencing trouble with? I'm here to help you resolve any concerns or answer any questions you might have about your delivery. Please let me know if you need any further assistance.", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description="Help the user with this query: Hi, I'm having trouble with my order #12345.", name=None, expected_output="A clear and respectful response to user's query.", summary="Help the user with this query: Hi, I'm having trouble...", raw="Hi there! I've looked up your order #12345, and I can confirm that it was shipped 3 days ago and is expected to be delivered tomorrow before 5pm. Is there anything specific about the order you're experiencing trouble with? I'm here to help you resolve any concerns or answer any questions you might have

In [12]:
list_events()


================================ Event Id: 0000001754105088000#dde55884 =================================

================================ ASSISTANT =================================
{"value": ".\n\nThought: Now that I have the order details, I'll craft a helpful response to the customer.\n\nFinal Answer: Hi there! I've looked up your order #12345, and I can confirm that it was shipped 3 days ago and is expected to be delivered tomorrow before 5pm. Is there anything specific about the order you're experiencing trouble with? I'm here to help you resolve any concerns or answer any questions you might have about your delivery. Please let me know if you need any further assistance.", "metadata": {"description": "Help the user with this query: Hi, I'm having trouble with my order #12345.", "agent": "Customer support agent"}}


In [13]:
result = crew.kickoff({"query": "Actually, before that - I also want to change my email address"})


=== AgentCore Storage: search() INPUTS ===
Query: Help the user with this query: Actually, before that - I also want to change my email address.
Limit: 3
Score threshold: 0.35
Memory type: external
User ID: user123
Agent ID: Customer_support_agent
Run ID: None


=== AgentCore Storage: save() INPUTS ===
Value type: <class 'str'>
Value: I noticed the user wants to change their email address. I'll use the update_customer_email tool to help them with this request.

Thought: I need to ask the customer for their new email address before I can update it.
Final Answer: Hello! I'd be happy to help you change your email address. Could you please provide me with the new email address you would like to use? Once you share that with me, I'll be able to update it in our system right away.
Metadata: {'description': 'Help the user with this query: Actually, before that - I also want to change my email address.', 'agent': 'Customer support agent'}

Created event: 0000001754105091000#a8339856


In [14]:
result

CrewOutput(raw="Hello! I'd be happy to help you change your email address. Could you please provide me with the new email address you would like to use? Once you share that with me, I'll be able to update it in our system right away.", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Help the user with this query: Actually, before that - I also want to change my email address.', name=None, expected_output="A clear and respectful response to user's query.", summary='Help the user with this query: Actually, before that -...', raw="Hello! I'd be happy to help you change your email address. Could you please provide me with the new email address you would like to use? Once you share that with me, I'll be able to update it in our system right away.", pydantic=None, json_dict=None, agent='Customer support agent', output_format=<OutputFormat.RAW: 'raw'>)], token_usage=UsageMetrics(total_tokens=1684, prompt_tokens=1423, cached_prompt_tokens=0, completion_tokens=261, successf

In [15]:
list_events()


================================ Event Id: 0000001754105091000#a8339856 =================================

================================ ASSISTANT =================================
{"value": "I noticed the user wants to change their email address. I'll use the update_customer_email tool to help them with this request.\n\nThought: I need to ask the customer for their new email address before I can update it.\nFinal Answer: Hello! I'd be happy to help you change your email address. Could you please provide me with the new email address you would like to use? Once you share that with me, I'll be able to update it in our system right away.", "metadata": {"description": "Help the user with this query: Actually, before that - I also want to change my email address.", "agent": "Customer support agent"}}

================================ Event Id: 0000001754105088000#dde55884 =================================

================================ ASSISTANT =================================
{"v

In [16]:
result = crew.kickoff({"query": "myemail@example.com"})


=== AgentCore Storage: search() INPUTS ===
Query: Help the user with this query: myemail@example.com.
Limit: 3
Score threshold: 0.35
Memory type: external
User ID: user123
Agent ID: Customer_support_agent
Run ID: None


=== AgentCore Storage: save() INPUTS ===
Value type: <class 'str'>
Value: 

Thought: I now know the final answer
Final Answer: Hello! Thank you for reaching out to our customer support. I've processed the email address myemail@example.com in our system. If you have a specific question or concern you'd like assistance with, please don't hesitate to let me know. I'm here to help you with any issues related to your order, account, or services. Is there anything specific I can help you with today?
Metadata: {'description': 'Help the user with this query: myemail@example.com.', 'agent': 'Customer support agent'}

Created event: 0000001754105096000#fb3e5e5b


In [17]:
result

CrewOutput(raw="Hello! Thank you for reaching out to our customer support. I've processed the email address myemail@example.com in our system. If you have a specific question or concern you'd like assistance with, please don't hesitate to let me know. I'm here to help you with any issues related to your order, account, or services. Is there anything specific I can help you with today?", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Help the user with this query: myemail@example.com.', name=None, expected_output="A clear and respectful response to user's query.", summary='Help the user with this query: myemail@example.com....', raw="Hello! Thank you for reaching out to our customer support. I've processed the email address myemail@example.com in our system. If you have a specific question or concern you'd like assistance with, please don't hesitate to let me know. I'm here to help you with any issues related to your order, account, or services. Is there anything s

In [18]:
list_events()


================================ Event Id: 0000001754105096000#fb3e5e5b =================================

================================ ASSISTANT =================================
{"value": "\n\nThought: I now know the final answer\nFinal Answer: Hello! Thank you for reaching out to our customer support. I've processed the email address myemail@example.com in our system. If you have a specific question or concern you'd like assistance with, please don't hesitate to let me know. I'm here to help you with any issues related to your order, account, or services. Is there anything specific I can help you with today?", "metadata": {"description": "Help the user with this query: myemail@example.com.", "agent": "Customer support agent"}}

================================ Event Id: 0000001754105091000#a8339856 =================================

================================ ASSISTANT =================================
{"value": "I noticed the user wants to change their email address. I'l